In [2]:
######## snakemake preamble start (automatically inserted, do not edit) ########
import sys

sys.path.extend(
    [
        "/Users/kamrantehranchi/opt/miniconda3/envs/pypsa-usa/lib/python3.10/site-packages/",
        "/Users/kamrantehranchi/Library/CloudStorage/OneDrive-Stanford/Kamran_OSW/PyPSA_Models/pypsa-breakthroughenergy-usa/workflow/notebooks/",
    ]
)
import pickle

# snakemake = pickle.loads(
#     b'\x80\x04\x95\xe1$\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c3results/usa/networks/elec_s_200_ec_lv1.1_Co2L0.5.nc\x94\x8c+resources/usa/regions_onshore_s_200.geojson\x94\x8c,resources/usa/regions_offshore_s_200.geojson\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\x07network\x94K\x00N\x86\x94\x8c\x0fregions_onshore\x94K\x01N\x86\x94\x8c\x10regions_offshore\x94K\x02N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\x18\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h\x1e)}\x94\x8c\x05_name\x94h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bh\x10h\nh\x12h\x0bh\x14h\x0cub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94(\x8c@results/usa/figures/elec_s_200_ec_lv1.1_Co2L0.5_capacity_map.pdf\x94\x8c@results/usa/figures/elec_s_200_ec_lv1.1_Co2L0.5_capacity_bar.pdf\x94\x8cAresults/usa/figures/elec_s_200_ec_lv1.1_Co2L0.5_operation_map.pdf\x94\x8cAresults/usa/figures/elec_s_200_ec_lv1.1_Co2L0.5_operation_bar.pdf\x94\x8cBresults/usa/figures/elec_s_200_ec_lv1.1_Co2L0.5_operation_area.pdf\x94\x8c<results/usa/figures/elec_s_200_ec_lv1.1_Co2L0.5_cost_bar.pdf\x94e}\x94(h\x0e}\x94(\x8c\x0ccapacity_map\x94K\x00N\x86\x94\x8c\x0ccapacity_bar\x94K\x01N\x86\x94\x8c\roperation_map\x94K\x02N\x86\x94\x8c\roperation_bar\x94K\x03N\x86\x94\x8c\x0eoperation_area\x94K\x04N\x86\x94\x8c\x08cost_bar\x94K\x05N\x86\x94uh\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bh4h,h6h-h8h.h:h/h<h0h>h1ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94}\x94(h\x0e}\x94h\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94(\x8c\x03usa\x94\x8c\x03200\x94\x8c\x04v1.1\x94\x8c\x07Co2L0.5\x94e}\x94(h\x0e}\x94(\x8c\x0cinterconnect\x94K\x00N\x86\x94\x8c\x08clusters\x94K\x01N\x86\x94\x8c\x02ll\x94K\x02N\x86\x94\x8c\x04opts\x94K\x03N\x86\x94uh\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94b\x8c\x0cinterconnect\x94h\\\x8c\x08clusters\x94h]\x8c\x02ll\x94h^\x8c\x04opts\x94h_ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x04/tmp\x94e}\x94(h\x0e}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bh\x7fK\x01h\x81K\x01h\x83h|ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94}\x94(h\x0e}\x94h\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bub\x8c\x06config\x94}\x94(\x8c\x07version\x94\x8c\x050.5.0\x94\x8c\x08tutorial\x94\x89\x8c\x07logging\x94}\x94(\x8c\x05level\x94\x8c\x04INFO\x94\x8c\x06format\x94\x8c"%(levelname)s:%(name)s:%(message)s\x94u\x8c\x08scenario\x94}\x94(\x8c\x05simpl\x94]\x94\x8c\x00\x94a\x8c\x02ll\x94]\x94\x8c\x04copt\x94a\x8c\x08clusters\x94]\x94(K%K\x80M\x00\x01M\x00\x02M\x00\x04e\x8c\x04opts\x94]\x94\x8c\x07Co2L-3H\x94au\x8c\tcountries\x94]\x94\x8c\x02US\x94a\x8c\tsnapshots\x94}\x94(\x8c\x05start\x94\x8c\n2013-01-01\x94\x8c\x03end\x94\x8c\n2014-01-01\x94\x8c\x06closed\x94\x8c\x04left\x94u\x8c\x06enable\x94}\x94(\x8c\x13prepare_links_p_nom\x94\x89\x8c\x13retrieve_databundle\x94\x88\x8c\x12retrieve_cost_data\x94\x88\x8c\x0cbuild_cutout\x94\x89\x8c\x0fretrieve_cutout\x94\x88\x8c\x13build_natura_raster\x94\x89\x8c\x16retrieve_natura_raster\x94\x88\x8c\rcustom_busmap\x94\x89u\x8c\x0belectricity\x94}\x94(\x8c\x08voltages\x94]\x94(G@l\xc0\x00\x00\x00\x00\x00G@u\x90\x00\x00\x00\x00\x00G@\x7f@\x00\x00\x00\x00\x00G@\x87\xe8\x00\x00\x00\x00\x00e\x8c\x08gaslimit\x94\x89\x8c\x08co2limit\x94GA\xd5\xf2I@\x00\x00\x00\x8c\x07co2base\x94GA\xdbn\xdb\x90\x00\x00\x00\x8c\x10agg_p_nom_limits\x94\x8c\x19data/agg_p_nom_minmax.csv\x94\x8c\x13operational_reserve\x94}\x94(\x8c\x08activate\x94\x89\x8c\x0cepsilon_load\x94G?\x94z\xe1G\xae\x14{\x8c\x0cepsilon_vres\x94G?\x94z\xe1G\xae\x14{\x8c\x0bcontingency\x94M\xa0\x0fu\x8c\tmax_hours\x94}\x94(\x8c\x07battery\x94K\x06\x8c\x02H2\x94K\xa8u\x8c\x13extendable_carriers\x94}\x94(\x8c\tGenerator\x94]\x94\x8c\x0bStorageUnit\x94]\x94\x8c\x05Store\x94]\x94(\x8c\x07battery\x94\x8c\x02H2\x94e\x8c\x04Link\x94]\x94\x8c\x0bH2 pipeline\x94au\x8c\x12powerplants_filter\x94\x8c\'(DateOut >= 2022 or DateOut != DateOut)\x94\x8c\x12custom_powerplants\x94\x89\x8c\x15conventional_carriers\x94]\x94(\x8c\x07nuclear\x94\x8c\x03oil\x94\x8c\x04OCGT\x94\x8c\x04CCGT\x94\x8c\x04coal\x94\x8c\x07lignite\x94\x8c\ngeothermal\x94\x8c\x07biomass\x94e\x8c\x12renewable_carriers\x94]\x94(\x8c\x05solar\x94\x8c\x06onwind\x94\x8c\noffwind-ac\x94\x8c\noffwind-dc\x94\x8c\x05hydro\x94e\x8c\x1destimate_renewable_capacities\x94}\x94(\x8c\x06enable\x94\x89\x8c\tfrom_opsd\x94\x88\x8c\x04year\x94M\xe4\x07\x8c\x0fexpansion_limit\x94\x89\x8c\x12technology_mapping\x94}\x94(\x8c\x08Offshore\x94]\x94(\x8c\noffwind-ac\x94\x8c\noffwind-dc\x94e\x8c\x07Onshore\x94]\x94\x8c\x06onwind\x94a\x8c\x02PV\x94]\x94\x8c\x05solar\x94auu\x8c\x12voltage_simplified\x94K\xe6u\x8c\x06atlite\x94}\x94(\x8c\nnprocesses\x94K\x04\x8c\x07cutouts\x94}\x94(\x8c\x10europe-2013-era5\x94}\x94(\x8c\x06module\x94\x8c\x04era5\x94\x8c\x01x\x94]\x94(G\xc0(\x00\x00\x00\x00\x00\x00G@A\x80\x00\x00\x00\x00\x00e\x8c\x01y\x94]\x94(G@@\x80\x00\x00\x00\x00\x00KHe\x8c\x02dx\x94G?\xd3333333\x8c\x02dy\x94G?\xd3333333\x8c\x04time\x94]\x94(\x8c\x042013\x94\x8c\x042013\x94eu\x8c\x11europe-2013-sarah\x94}\x94(\x8c\x06module\x94]\x94(\x8c\x05sarah\x94\x8c\x04era5\x94ej\x1a\x01\x00\x00]\x94(G\xc0(\x00\x00\x00\x00\x00\x00G@F\x80\x00\x00\x00\x00\x00ej\x1c\x01\x00\x00]\x94(G@@\x80\x00\x00\x00\x00\x00KAe\x8c\x02dx\x94G?\xc9\x99\x99\x99\x99\x99\x9a\x8c\x02dy\x94G?\xc9\x99\x99\x99\x99\x99\x9a\x8c\x04time\x94]\x94(\x8c\x042013\x94\x8c\x042013\x94e\x8c\x11sarah_interpolate\x94\x89\x8c\tsarah_dir\x94N\x8c\x08features\x94]\x94(\x8c\x06influx\x94\x8c\x0btemperature\x94euuu\x8c\trenewable\x94}\x94(\x8c\x06onwind\x94}\x94(\x8c\x06cutout\x94\x8c\x10europe-2013-era5\x94\x8c\x08resource\x94}\x94(\x8c\x06method\x94\x8c\x04wind\x94\x8c\x07turbine\x94\x8c\x0fVestas_V112_3MW\x94u\x8c\x11capacity_per_sqkm\x94K\x03\x8c\x06corine\x94}\x94(\x8c\ngrid_codes\x94]\x94(K\x0cK\rK\x0eK\x0fK\x10K\x11K\x12K\x13K\x14K\x15K\x16K\x17K\x18K\x19K\x1aK\x1bK\x1cK\x1dK\x1fK e\x8c\x08distance\x94M\xe8\x03\x8c\x13distance_grid_codes\x94]\x94(K\x01K\x02K\x03K\x04K\x05K\x06eu\x8c\x06natura\x94\x88\x8c\x13excluder_resolution\x94Kd\x8c\tpotential\x94\x8c\x06simple\x94\x8c\rclip_p_max_pu\x94G?\x84z\xe1G\xae\x14{u\x8c\noffwind-ac\x94}\x94(\x8c\x06cutout\x94\x8c\x10europe-2013-era5\x94\x8c\x08resource\x94}\x94(\x8c\x06method\x94\x8c\x04wind\x94\x8c\x07turbine\x94\x8c"NREL_ReferenceTurbine_5MW_offshore\x94u\x8c\x11capacity_per_sqkm\x94K\x02\x8c\x11correction_factor\x94G?\xecV\x04\x18\x93t\xbc\x8c\x06corine\x94]\x94(K,K\xffe\x8c\x06natura\x94\x88\x8c\x0eship_threshold\x94M\x90\x01\x8c\tmax_depth\x94K2\x8c\x12max_shore_distance\x94M0u\x8c\x13excluder_resolution\x94K\xc8\x8c\tpotential\x94\x8c\x06simple\x94\x8c\rclip_p_max_pu\x94G?\x84z\xe1G\xae\x14{u\x8c\noffwind-dc\x94}\x94(\x8c\x06cutout\x94\x8c\x10europe-2013-era5\x94\x8c\x08resource\x94}\x94(\x8c\x06method\x94\x8c\x04wind\x94\x8c\x07turbine\x94\x8c"NREL_ReferenceTurbine_5MW_offshore\x94u\x8c\x11capacity_per_sqkm\x94K\x02\x8c\x11correction_factor\x94G?\xecV\x04\x18\x93t\xbc\x8c\x06corine\x94]\x94(K,K\xffe\x8c\x06natura\x94\x88\x8c\x0eship_threshold\x94M\x90\x01\x8c\tmax_depth\x94K2\x8c\x12min_shore_distance\x94M0u\x8c\x13excluder_resolution\x94K\xc8\x8c\tpotential\x94\x8c\x06simple\x94\x8c\rclip_p_max_pu\x94G?\x84z\xe1G\xae\x14{u\x8c\x05solar\x94}\x94(\x8c\x06cutout\x94\x8c\x11europe-2013-sarah\x94\x8c\x08resource\x94}\x94(\x8c\x06method\x94\x8c\x02pv\x94\x8c\x05panel\x94\x8c\x03CSi\x94\x8c\x0borientation\x94}\x94(\x8c\x05slope\x94G@A\x80\x00\x00\x00\x00\x00\x8c\x07azimuth\x94G@f\x80\x00\x00\x00\x00\x00uu\x8c\x11capacity_per_sqkm\x94G?\xfb333333\x8c\x06corine\x94]\x94(K\x01K\x02K\x03K\x04K\x05K\x06K\x07K\x08K\tK\nK\x0bK\x0cK\rK\x0eK\x0fK\x10K\x11K\x12K\x13K\x14K\x1aK\x1fK e\x8c\x06natura\x94\x88\x8c\x13excluder_resolution\x94Kd\x8c\tpotential\x94\x8c\x06simple\x94\x8c\rclip_p_max_pu\x94G?\x84z\xe1G\xae\x14{u\x8c\x05hydro\x94}\x94(\x8c\x06cutout\x94\x8c\x10europe-2013-era5\x94\x8c\x08carriers\x94]\x94(\x8c\x03ror\x94\x8c\x03PHS\x94\x8c\x05hydro\x94e\x8c\rPHS_max_hours\x94K\x06\x8c\x0fhydro_max_hours\x94\x8c!energy_capacity_totals_by_country\x94\x8c\x0fclip_min_inflow\x94G?\xf0\x00\x00\x00\x00\x00\x00\x8c\tpotential\x94\x8c\x06simple\x94u\x8c\x04wind\x94}\x94\x8c\tpotential\x94\x8c\x06simple\x94su\x8c\x0cconventional\x94}\x94\x8c\x07nuclear\x94}\x94\x8c\x08p_max_pu\x94\x8c\x19data/nuclear_p_max_pu.csv\x94ss\x8c\x05lines\x94}\x94(\x8c\x05types\x94}\x94(G@k\x80\x00\x00\x00\x00\x00\x8c\x1bAl/St 240/40 2-bundle 220.0\x94G@r\xc0\x00\x00\x00\x00\x00\x8c\x1bAl/St 240/40 3-bundle 300.0\x94G@w\xc0\x00\x00\x00\x00\x00\x8c\x1bAl/St 240/40 4-bundle 380.0\x94u\x8c\x08s_max_pu\x94G?\xe6ffffff\x8c\ts_nom_max\x94G\x7f\xf0\x00\x00\x00\x00\x00\x00\x8c\rlength_factor\x94G?\xf4\x00\x00\x00\x00\x00\x00\x8c\x12under_construction\x94\x8c\x04zero\x94u\x8c\x05links\x94}\x94(\x8c\x08p_max_pu\x94G?\xf0\x00\x00\x00\x00\x00\x00\x8c\tp_nom_max\x94G\x7f\xf0\x00\x00\x00\x00\x00\x00\x8c\rinclude_tyndp\x94\x88\x8c\x12under_construction\x94\x8c\x04zero\x94u\x8c\x0ctransformers\x94}\x94(j\x1a\x01\x00\x00G?\xb9\x99\x99\x99\x99\x99\x9a\x8c\x05s_nom\x94G@\x9f@\x00\x00\x00\x00\x00\x8c\x04type\x94h\xacu\x8c\x04load\x94}\x94(\x8c\x10power_statistics\x94\x88\x8c\x11interpolate_limit\x94K\x03\x8c\x19time_shift_for_large_gaps\x94\x8c\x021w\x94\x8c\x12manual_adjustments\x94\x88\x8c\x0escaling_factor\x94G?\xf0\x00\x00\x00\x00\x00\x00u\x8c\x05costs\x94}\x94(\x8c\x04year\x94M\xee\x07\x8c\x07version\x94\x8c\x06v0.4.0\x94\x8c\rrooftop_share\x94G?\xc1\xeb\x85\x1e\xb8Q\xec\x8c\x0bfill_values\x94}\x94(\x8c\x03FOM\x94K\x00\x8c\x03VOM\x94K\x00\x8c\nefficiency\x94K\x01\x8c\x04fuel\x94K\x00\x8c\ninvestment\x94K\x00\x8c\x08lifetime\x94K\x19\x8c\rCO2 intensity\x94K\x00\x8c\rdiscount rate\x94G?\xb1\xeb\x85\x1e\xb8Q\xecu\x8c\rmarginal_cost\x94}\x94(\x8c\x05solar\x94G?\x84z\xe1G\xae\x14{\x8c\x06onwind\x94G?\x8e\xb8Q\xeb\x85\x1e\xb8\x8c\x07offwind\x94G?\x8e\xb8Q\xeb\x85\x1e\xb8\x8c\x05hydro\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x02H2\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x0celectrolysis\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\tfuel cell\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x07battery\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x10battery inverter\x94G\x00\x00\x00\x00\x00\x00\x00\x00u\x8c\x0femission_prices\x94}\x94\x8c\x03co2\x94G\x00\x00\x00\x00\x00\x00\x00\x00su\x8c\nclustering\x94}\x94(\x8c\x10simplify_network\x94}\x94(\x8c\x0eto_substations\x94\x89\x8c\talgorithm\x94\x8c\x06kmeans\x94\x8c\x07feature\x94\x8c\x11solar+onwind-time\x94u\x8c\x0fcluster_network\x94}\x94(\x8c\talgorithm\x94\x8c\x06kmeans\x94\x8c\x07feature\x94\x8c\x11solar+onwind-time\x94u\x8c\x16aggregation_strategies\x94}\x94\x8c\ngenerators\x94}\x94(\x8c\tp_nom_max\x94\x8c\x03sum\x94\x8c\tp_nom_min\x94\x8c\x03sum\x94\x8c\x08p_min_pu\x94\x8c\x04mean\x94\x8c\rmarginal_cost\x94\x8c\x04mean\x94\x8c\x0bcommittable\x94\x8c\x03any\x94\x8c\rramp_limit_up\x94\x8c\x03max\x94\x8c\x0framp_limit_down\x94\x8c\x03max\x94\x8c\nefficiency\x94\x8c\x04mean\x94usu\x8c\x07solving\x94}\x94(\x8c\x07options\x94}\x94(\x8c\x0bformulation\x94\x8c\tkirchhoff\x94\x8c\rload_shedding\x94\x89\x8c\x0bnoisy_costs\x94\x88\x8c\x0emin_iterations\x94K\x04\x8c\x0emax_iterations\x94K\x06\x8c\rclip_p_max_pu\x94G?\x84z\xe1G\xae\x14{\x8c\x0fskip_iterations\x94\x88\x8c\x10track_iterations\x94\x89u\x8c\x06solver\x94}\x94(\x8c\x04name\x94\x8c\x06gurobi\x94\x8c\x07threads\x94K\x04\x8c\x06method\x94K\x02\x8c\tcrossover\x94K\x00\x8c\nBarConvTol\x94G>\xe4\xf8\xb5\x88\xe3h\xf1\x8c\x0eFeasibilityTol\x94G>\xb0\xc6\xf7\xa0\xb5\xed\x8d\x8c\x07AggFill\x94K\x00\x8c\x07PreDual\x94K\x00\x8c\x17GURO_PAR_BARDENSETHRESH\x94K\xc8uu\x8c\x08plotting\x94}\x94(\x8c\x03map\x94}\x94(\x8c\x07figsize\x94]\x94(K\x07K\x07e\x8c\nboundaries\x94]\x94(G\xc0$ffffffK\x1dK#KHe\x8c\x05p_nom\x94}\x94(\x8c\x0fbus_size_factor\x94G@\xe8j\x00\x00\x00\x00\x00\x8c\x10linewidth_factor\x94G@\xa7p\x00\x00\x00\x00\x00uu\x8c\tcosts_max\x94M \x03\x8c\x0fcosts_threshold\x94K\x01\x8c\nenergy_max\x94G@\xcdL\x00\x00\x00\x00\x00\x8c\nenergy_min\x94G\xc0\xc3\x88\x00\x00\x00\x00\x00\x8c\x10energy_threshold\x94G@I\x00\x00\x00\x00\x00\x00\x8c\tvre_techs\x94]\x94(\x8c\x06onwind\x94\x8c\noffwind-ac\x94\x8c\noffwind-dc\x94\x8c\x05solar\x94\x8c\x03ror\x94e\x8c\nconv_techs\x94]\x94(\x8c\x04OCGT\x94\x8c\x04CCGT\x94\x8c\x07Nuclear\x94\x8c\x04Coal\x94e\x8c\rstorage_techs\x94]\x94(\x8c\thydro+PHS\x94\x8c\x07battery\x94\x8c\x02H2\x94e\x8c\rload_carriers\x94]\x94\x8c\x07AC load\x94a\x8c\x0bAC_carriers\x94]\x94(\x8c\x07AC line\x94\x8c\x0eAC transformer\x94e\x8c\rlink_carriers\x94]\x94(\x8c\x07DC line\x94\x8c\x0fConverter AC-DC\x94e\x8c\x0btech_colors\x94}\x94(\x8c\x06onwind\x94\x8c\x07#235ebc\x94\x8c\x0conshore wind\x94\x8c\x07#235ebc\x94\x8c\x07offwind\x94\x8c\x07#6895dd\x94\x8c\noffwind-ac\x94\x8c\x07#6895dd\x94\x8c\roffshore wind\x94\x8c\x07#6895dd\x94\x8c\x10offshore wind ac\x94\x8c\x07#6895dd\x94\x8c\noffwind-dc\x94\x8c\x07#74c6f2\x94\x8c\x10offshore wind dc\x94\x8c\x07#74c6f2\x94\x8c\x05hydro\x94\x8c\x07#08ad97\x94\x8c\thydro+PHS\x94\x8c\x07#08ad97\x94\x8c\x03PHS\x94\x8c\x07#08ad97\x94\x8c\x0fhydro reservoir\x94\x8c\x07#08ad97\x94\x8c\x10hydroelectricity\x94\x8c\x07#08ad97\x94\x8c\x03ror\x94\x8c\x07#4adbc8\x94\x8c\x0crun of river\x94\x8c\x07#4adbc8\x94\x8c\x05solar\x94\x8c\x07#f9d002\x94\x8c\x08solar PV\x94\x8c\x07#f9d002\x94\x8c\rsolar thermal\x94\x8c\x07#ffef60\x94\x8c\x07biomass\x94\x8c\x07#0c6013\x94\x8c\rsolid biomass\x94\x8c\x07#06540d\x94\x8c\x06biogas\x94\x8c\x07#23932d\x94\x8c\x05waste\x94\x8c\x07#68896b\x94\x8c\ngeothermal\x94\x8c\x07#ba91b1\x94\x8c\x04OCGT\x94\x8c\x07#d35050\x94\x8c\x03gas\x94\x8c\x07#d35050\x94\x8c\x0bnatural gas\x94\x8c\x07#d35050\x94\x8c\x04CCGT\x94\x8c\x07#b20101\x94\x8c\x07nuclear\x94\x8c\x07#ff9000\x94\x8c\x04coal\x94\x8c\x07#707070\x94\x8c\x07lignite\x94\x8c\x07#9e5a01\x94\x8c\x03oil\x94\x8c\x07#262626\x94\x8c\x02H2\x94\x8c\x07#ea048a\x94\x8c\x10hydrogen storage\x94\x8c\x07#ea048a\x94\x8c\x07battery\x94\x8c\x07#b8ea04\x94\x8c\rElectric load\x94\x8c\x07#f9d002\x94\x8c\x0belectricity\x94\x8c\x07#f9d002\x94\x8c\x05lines\x94\x8c\x07#70af1d\x94\x8c\x12transmission lines\x94\x8c\x07#70af1d\x94\x8c\x05AC-AC\x94\x8c\x07#70af1d\x94\x8c\x07AC line\x94\x8c\x07#70af1d\x94\x8c\x05links\x94\x8c\x07#8a1caf\x94\x8c\nHVDC links\x94\x8c\x07#8a1caf\x94\x8c\x05DC-DC\x94\x8c\x07#8a1caf\x94\x8c\x07DC link\x94\x8c\x07#8a1caf\x94u\x8c\nnice_names\x94}\x94(\x8c\x04OCGT\x94\x8c\x0eOpen-Cycle Gas\x94\x8c\x04CCGT\x94\x8c\x12Combined-Cycle Gas\x94\x8c\noffwind-ac\x94\x8c\x12Offshore Wind (AC)\x94\x8c\noffwind-dc\x94\x8c\x12Offshore Wind (DC)\x94\x8c\x06onwind\x94\x8c\x0cOnshore Wind\x94\x8c\x05solar\x94\x8c\x05Solar\x94\x8c\x03PHS\x94\x8c\x14Pumped Hydro Storage\x94\x8c\x05hydro\x94\x8c\x0fReservoir & Dam\x94\x8c\x07battery\x94\x8c\x0fBattery Storage\x94\x8c\x02H2\x94\x8c\x10Hydrogen Storage\x94\x8c\x05lines\x94\x8c\x12Transmission Lines\x94\x8c\x03ror\x94\x8c\x0cRun of River\x94uu\x8c\tscenarios\x94}\x94(\x8c\x03all\x94}\x94(\x8c\x0cinterconnect\x94\x8c\x03usa\x94\x8c\x08clusters\x94K\xc8\x8c\x04opts\x94]\x94(\x8c\x07Co2L0.0\x94\x8c\x07Co2L0.1\x94\x8c\x07Co2L0.2\x94\x8c\x07Co2L0.3\x94\x8c\x07Co2L0.4\x94\x8c\x07Co2L0.5\x94\x8c\x07Co2L0.6\x94\x8c\x07Co2L0.7\x94\x8c\x07Co2L0.8\x94\x8c\x07Co2L0.9\x94\x8c\x07Co2L1.0\x94e\x8c\x02ll\x94]\x94(\x8c\x04vopt\x94\x8c\x04v1.0\x94\x8c\x04v1.1\x94\x8c\x04v1.2\x94eu\x8c\x04test\x94}\x94(\x8c\x0cinterconnect\x94\x8c\x05texas\x94\x8c\x08clusters\x94K\x05\x8c\x04opts\x94]\x94(\x8c\x07Co2L0.0\x94\x8c\x07Co2L0.5\x94\x8c\x07Co2L1.0\x94e\x8c\x02ll\x94\x8c\x04v1.0\x94uu\x8c\x0bsubworkflow\x94\x8c\npypsa-eur/\x94\x8c\x11zenodo_repository\x94}\x94(\x8c\x03url\x94\x8c9https://zenodo.org/record/4538590/files/USATestSystem.zip\x94\x8c\x03use\x94\x89u\x8c\x10allowed_carriers\x94]\x94(\x8c\x04wind\x94\x8c\x05solar\x94\x8c\x07offwind\x94\x8c\x05hydro\x94\x8c\x04coal\x94\x8c\x07nuclear\x94\x8c\x02ng\x94\x8c\x03oil\x94\x8c\ngeothermal\x94e\x8c\x14committable_carriers\x94]\x94\x8c\x13extendable_carriers\x94]\x94(\x8c\x05solar\x94\x8c\x04wind\x94\x8c\x07offwind\x94e\x8c\x07cluster\x94\x8c=gridlock:/home/vres/data/fabian/pypsa-breakthroughenergy-usa/\x94u\x8c\x04rule\x94\x8c\x13plot_figures_single\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8cD/home/fabian/vres/py/pypsa-breakthroughenergy-usa/workflow/notebooks\x94ub.'
# )

from snakemake.logging import logger

logger.printshellcmds = False
import os

os.chdir(r"/Users/kamrantehranchi/Library/CloudStorage/OneDrive-Stanford/Kamran_OSW/PyPSA_Models/pypsa-breakthroughenergy-usa/workflow");
######## snakemake preamble end #########

In [ ]:
# start coding here

In [3]:
import pypsa
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from datetime import datetime
from cartopy import crs as ccrs
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches

/Users/kamrantehranchi/opt/miniconda3/envs/pypsa-usa/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning:

The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.



In [4]:
sns.set_theme("paper", style="darkgrid")

In [5]:
n = pypsa.Network(snakemake.input.network)
regions_onshore = gpd.read_file(snakemake.input.regions_onshore)

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/kamrantehranchi/Library/CloudStorage/OneDrive-Stanford/Kamran_OSW/PyPSA_Models/pypsa-breakthroughenergy-usa/results/usa/networks/elec_s_200_ec_lv1.1_Co2L0.5.nc'

In [ ]:
# TODO
n.carriers.loc["wind", "color"] = "lightblue"
n.carriers.loc["wind_offshore", "color"] = "lightblue"
n.carriers.loc["wind_offshore", "nice_name"] = "Offshore Wind"
n.carriers.loc["ng", "color"] = "indianred"
n.carriers.loc["dfo", "color"] = "k"
n.carriers.loc["other", "color"] = "grey"

In [ ]:
# TODO
generating_link_carrier_map = {"fuel cell": "H2", "battery discharger": "battery"}

# Capacity plots

In [ ]:
if snakemake.wildcards.interconnect != "usa":
    bus_scale = 1e5
else:
    bus_scale = 4e4

if snakemake.wildcards.interconnect != "usa":
    line_scale = 2e3
else:
    line_scale = 4e3

In [ ]:
fig, ax = plt.subplots(
    figsize=(8, 8), subplot_kw={"projection": ccrs.EqualEarth(n.buses.x.mean())}
)
g = n.generators.groupby(["bus", "carrier"]).p_nom_opt.sum()
s = (
    n.links.query("carrier in @generating_link_carrier_map")
    .groupby(["bus1", "carrier"])
    .p_nom_opt.sum()
)
s = s.rename(index=generating_link_carrier_map, level=1)
buses = pd.concat([g, s])
with plt.rc_context({"patch.linewidth": 0.1}):
    n.plot(
        bus_sizes=buses / bus_scale,
        bus_alpha=0.7,
        line_widths=n.lines.s_nom_opt / line_scale,
        link_widths=n.links.p_nom_opt / line_scale,
        line_colors="teal",
        ax=ax,
        margin=0.2,
        color_geomap=None,
    )
regions_onshore.plot(
    ax=ax,
    facecolor="whitesmoke",
    edgecolor="white",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=1.2,
)
ax.set_extent(regions_onshore.total_bounds[[0, 2, 1, 3]])


legend_kwargs = {"loc": "upper left", "frameon": False}
bus_sizes = [5000, 10e3, 50e3]  # in MW
line_sizes = [2000, 5000]  # in MW
add_legend_circles(
    ax,
    [s / bus_scale for s in bus_sizes],
    [f"{s / 1000} GW" for s in bus_sizes],
    legend_kw={"bbox_to_anchor": (1, 1), **legend_kwargs},
)
add_legend_lines(
    ax,
    [s / line_scale for s in line_sizes],
    [f"{s / 1000} GW" for s in line_sizes],
    legend_kw={"bbox_to_anchor": (1, 0.8), **legend_kwargs},
)
add_legend_patches(
    ax,
    n.carriers.color,
    n.carriers.nice_name,
    legend_kw={"bbox_to_anchor": (1, 0), **legend_kwargs, "loc": "lower left"},
)
fig.tight_layout()
fig.savefig(snakemake.output.capacity_map)

In [ ]:
fig, ax = plt.subplots()
capacities = n.generators.groupby("carrier").p_nom_opt.sum()
capacities.rename(n.carriers.nice_name, inplace=True)
colors = n.carriers.set_index("nice_name").color[capacities.index]
capacities.div(1e3).plot.bar(color=colors, ax=ax)
ax.set_ylabel("Total capacity [GW]")
ax.set_xlabel("")
fig.savefig(snakemake.output.capacity_bar)

# Operation plots

In [ ]:
if snakemake.wildcards.interconnect != "usa":
    bus_scale = 5e7
else:
    bus_scale = 5e7

if snakemake.wildcards.interconnect != "usa":
    line_scale = 1e6
else:
    line_scale = 1e7

In [ ]:
fig, ax = plt.subplots(
    figsize=(8, 8), subplot_kw={"projection": ccrs.EqualEarth(n.buses.x.mean())}
)
with plt.rc_context({"patch.linewidth": 0.1}):
    n.plot(
        bus_sizes=n.generators_t.p.sum()
        .groupby([n.generators.bus, n.generators.carrier])
        .sum()
        / bus_scale,
        bus_alpha=0.7,
        line_widths=n.lines_t.p0.sum() / line_scale,
        link_widths=n.links_t.p0.sum() / line_scale,
        line_colors="teal",
        ax=ax,
        margin=0.2,
        color_geomap=None,
    )
regions_onshore.plot(
    ax=ax,
    facecolor="whitesmoke",
    edgecolor="white",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=1.2,
)
ax.set_extent(regions_onshore.total_bounds[[0, 2, 1, 3]])


legend_kwargs = {"loc": "upper left", "frameon": False}
bus_sizes = [1000000, 2000000, 5000000]  # in MW
line_sizes = [2000000, 5000000]  # in MW
add_legend_circles(
    ax,
    [s / bus_scale for s in bus_sizes],
    [f"{s / 1e6} TWh" for s in bus_sizes],
    legend_kw={"bbox_to_anchor": (1, 1), **legend_kwargs},
)
add_legend_lines(
    ax,
    [s / line_scale for s in line_sizes],
    [f"{s / 1e6} TWh" for s in line_sizes],
    legend_kw={"bbox_to_anchor": (1, 0.8), **legend_kwargs},
)
add_legend_patches(
    ax,
    n.carriers.color,
    n.carriers.nice_name,
    legend_kw={"bbox_to_anchor": (1, 0.0), **legend_kwargs, "loc": "lower left"},
)
fig.tight_layout()
fig.savefig(snakemake.output.operation_map)

# Cost plots

In [ ]:
carriers = n.generators.carrier
production = (
    n.generators_t.p.groupby(carriers, axis=1)
    .sum()
    .rename(columns=n.carriers.nice_name)
    / 1e3
)
production = production.loc[:, production.sum() > 0.1]
demand = n.loads_t.p.sum(1).rename("Demand") / 1e3
colors = n.carriers.set_index("nice_name").color[production.columns]

if snakemake.config['solving']['options']['nhours'] < 8760:
    nhours= snakemake.config['solving']['options']['nhours']
    enddate = pd.to_datetime('2016-01-01') + pd.Timedelta(nhours%24,'h')
    enddate
    for timeslice in list(range(1, enddate.month)) + ["all"]:
        snapshots = (
            n.snapshots.get_loc(f"2016-{timeslice}")
            if timeslice != "all"
            else slice(None, None)
        )
        fig, ax = plt.subplots(figsize=(14, 4))
        production[snapshots].plot.area(ax=ax, color=colors, alpha=0.7, legend="reverse")
        # demand.plot.line(ax=ax, ls='-', color='darkblue')
        ax.legend(bbox_to_anchor=(1, 1), loc="upper left")
        ax.set_ylabel("Power [GW]")
        ax.set_xlabel("")
        fig.tight_layout()
        suffix = (
            "-" + datetime.strptime(str(timeslice), "%m").strftime("%b")
            if timeslice != "all"
            else ""
        )
        path = Path(snakemake.output.operation_area)
        fig.savefig(path.parent / (path.stem + suffix + path.suffix))

else:
    for timeslice in list(range(1, 12)) + ["all"]:
        snapshots = (
            n.snapshots.get_loc(f"2016-{timeslice}")
            if timeslice != "all"
            else slice(None, None)
        )
        fig, ax = plt.subplots(figsize=(14, 4))
        production[snapshots].plot.area(ax=ax, color=colors, alpha=0.7, legend="reverse")
        # demand.plot.line(ax=ax, ls='-', color='darkblue')
        ax.legend(bbox_to_anchor=(1, 1), loc="upper left")
        ax.set_ylabel("Power [GW]")
        ax.set_xlabel("")
        fig.tight_layout()
        suffix = (
            "-" + datetime.strptime(str(timeslice), "%m").strftime("%b")
            if timeslice != "all"
            else ""
        )
        path = Path(snakemake.output.operation_area)
        fig.savefig(path.parent / (path.stem + suffix + path.suffix))

In [ ]:
fig, ax = plt.subplots()
total_production = n.snapshot_weightings.generators @ production
total_production.div(1e3).plot.bar(color=colors, ax=ax)
ax.set_ylabel("Total production [TWh]")
ax.set_xlabel("")
fig.savefig(snakemake.output.operation_bar)

In [ ]:
fig, ax = plt.subplots()

production = n.generators_t.p
operational_costs = (
    (production * n.generators.marginal_cost)
    .groupby(carriers, axis=1)
    .sum()
    .rename(columns=n.carriers.nice_name)
).sum()

capital_costs = (
    n.generators.eval("p_nom_opt * capital_cost")
    .groupby(carriers)
    .sum()
    .rename(n.carriers.nice_name)
)

costs = pd.concat([operational_costs, capital_costs], axis=1, keys=["OPEX", "CAPEX"])
costs = costs.reset_index("carrier")


s1 = sns.barplot(y="carrier", x="CAPEX", data=costs, alpha=0.6, ax=ax, palette=colors)
s2 = sns.barplot(
    y="carrier", x="OPEX", data=costs, ax=ax, left=costs["CAPEX"], palette=colors
)

ax.set_ylabel("")
ax.set_xlabel("CAPEX & OPEX [bn€]")
fig.savefig(snakemake.output.cost_bar)